# **1、赛题简介**

机器阅读理解(Machine Reading Comprehension)是指让机器阅读文本，然后回答和阅读内容相关的问题。阅读理解是自然语言处理和人工智能领域的重要前沿课题，对于提升机器智能水平、使机器具有持续知识获取能力具有重要价值，近年来受到学术界和工业界的广泛关注。

本次竞赛设立了面向观点型问题的机器阅读理解任务，旨在为研究者提供开放的学术交流平台，进一步提升阅读理解的研究水平，推动语言理解和人工智能领域技术和应用的发展。面向观点型问题的机器阅读理解源于真实的应用场景：在智能搜索问答等产品中，针对用户输入的观点型问题，搜索引擎首先会检索相关候选文档，然后从候选文档中抽取出能够回答用户问题的答案段落摘要，最后给出答案段落摘要所包含的是非观点。本次任务聚焦于预测答案段落摘要中所表述的是非观点极性。

# **2、评价指标**

竞赛基于测试集人工标注的观点答案，采用答案分类的准确率（Accuracy）作为评价指标：![](https://ai-studio-static-online.cdn.bcebos.com/7e24586a5427447780dfd598bf417b94a739b7b890324fa3a728528aa55ec3a7)


# **3、竞赛数据范围**

数据集范围：所有问题均为搜索引擎中按照用户行为分布的观点类的问题，对应的文档为搜索引擎搜索出来的网页，答案片段摘要是经过人工标注的回答此问题的答案片段，是非观点极性是此答案片段摘要相对于问题的观点极性。
是非观点极性：所有极性均在{“ Yes”, “No“, “Depends“}集合中，对应的含义为：
Yes：肯定观点，肯定观点指的是答案给出了较为明确的肯定态度。有客观事实的从客观事实的角度出发，主观态度类的从答案的整体态度来判断。
No：否定观点，否定观点通常指的是答案较为明确的给出了与问题相反的态度。
Depends：无法确定/分情况，主要指的是事情本身存在多种情况，不同情况下对应的观点不一致；或者答案本身对问题表示不确定，要具体具体情况才能判断。

在本次比赛中，我们使用了目前效果比较好的预训练模型，分别是 RoBERTa， ERNIE，BERT，作为我们的分类模型，首先我们将处理好的语料分别输入到三个 模型，在预训练的基础上进行训练，将训练好的模型对测试集预测，得到测试集 的概率分布，之后将三个模型所得到的概率文件进行融合，在本次比赛中我们采 用两种融合方式，一种是使用树模型进行融合，一种是手工调参进行融合。在实 际的比赛过程中，由选手进行手工调参的融合效果好于使用树模型进行融合，其 原因在于树模型容易过拟合。之后使用分别针对三个模型使用 k 折法，分数从 82 提升到 82.8。
**关键词:预训练模型，k 折，融合**

# **4、思路**

通过分析发现比赛的任务可以转化为自然语言处理中的文本分类任务，将问 题和答案一同输入给模型，从而判断问答对的所属类别。在比赛初期，我们使用 比赛主办方所提供的开源代码进行分类，但是在比赛过程中我们发现官方的代码 并不适合本次比赛，主要原因有以下几点:
(1) 这次比赛时间相比传统比赛时间过于短，而官方所提供代码过于严谨，不易 改动导致比赛过程中，细小的改动即可造成全局代码的崩盘，所以在时间比 较紧的比赛中是不适合使用这种代码。
(2) 代码所依赖的paddlepaddle框架在1.5对于目前1paddlepaddle1.8的框架来 说是相对于较为陈旧的。
(3) 不易于做k折法。开源代码中使用py_reader函数进行语料的读取，不方便 进行k折。
![](https://ai-studio-static-online.cdn.bcebos.com/80bbebfb099c4ff3a030d621ba207aab92db47fe5e2446c7828e38fe37cf80ce)


在解决这个问题的同时，我们也确定了使用的模型，RoBERTa，BERT, ERNIE。我们采取在已经预训练好的模型上面接一层全连接层从而进行微调得 到预测结果。之后我们将所得到三个模型 k 折所得到的概率进行融合。
![](https://ai-studio-static-online.cdn.bcebos.com/226c7fe0ee304d02b99e4a9b48eabee7800c672c90e64834bf256512061cfe0c)


# **5、实验流程**
首先是数据集的处理，赛题所提供的数据为JSON格式，无法直接输入到PALM框架中，所以先对数据集进行拆分，转换成textA、textB、Label这样的格式，最后以tsv文件格式保存，以1、2、0分别代表Yes、Depends、No
```
{"answer": "这种疾病在临床上是可以治好的!但是一定要坚持早发现早治疗的原则，否则病情一旦加重，会有生命危险，治疗起来也相对困难。", "yesno_answer": "Yes", "documents": [{"paragraphs": ["强直性脊柱炎是较为常见的疾病,而且是一种全身性的疾病,累积到整个脊柱,会影响到髋关节,病程缠绵,且易造成残疾。因此强直性脊柱炎应及早治疗,积极控制病情发展,预防和避免脊椎变形。强直性脊柱炎在没有变形之前是可以治疗的,如果变形就只能控制不发展.早中期的患者建议采用中医系统化的调理方法,从病因上治疗强直性脊柱炎。治疗用纯中药追风舒筋疗法,没有任何副作用,经济实惠,疗效好,疗程短。不要用西药,副作用太大. 对于中晚期患者,关节功能受到限制,这样在治疗,一旦患者关节融合,在经过治疗也是不能达到功能恢复的目的。 患者应该了解的是,强直性脊柱炎本身就是一种世界性疾病,治疗上难度大,且患者需要长期坚持治疗,才能控制病情的发展. 对于强直性脊柱炎的用药治疗,主要是保守治疗,中药治疗、西药治疗、生物制"], "is_selected": true, "title": "强直性脊柱炎可以治好吗?_百度知道"}, {"paragraphs": ["得了强直性脊柱炎 能治好吗?很多强直性脊柱炎患者的心里有这样的疑问,专家告诉您,这种疾病在临床上是可以治好的!但是一定要坚持早发现早治疗的原则,否则病情一旦加重,会有生命危险,治疗起来也相对困难。 首先,我们来了解一下什么是强直性脊柱炎?强直性脊柱炎是不常见风湿免疫性疾病,是血清阴性关节病的一种,研究表明,该病原因尚不很明确,以脊柱为主要病变的慢性疾病,病变主要累及骶髂关节,引起脊柱强直和纤维化,造成弯腰、行走活动受限,并可有不同程度的眼、肺、肌肉、骨骼的病变,也有自身免疫功能的紊乱,所以又属自身免疫性疾病。 一般而言,强直性脊柱炎在临床上主要有如下几种表征:腰背痛、晨僵、腰椎各方向活动受限和胸廓活动度减低是强直性脊柱炎的典型表现,在病情活动期间表现更为明显。通常来说,强直性脊柱"], "is_selected": true, "title": "得了强直性脊柱炎真的能治好吗_百度知道"}, {"paragraphs": ["到底强直性脊柱炎能治好吗_临床医学_医药卫生_专业资料。到底强直性脊柱炎能治好吗?强直性脊柱炎是一种慢性炎症性疾病,在当今的临床上面控制起来的难度很大的,首先侵犯骶髂关节。早期病变处会出现炎症性的疼痛,伴随着肌肉痉挛等的症状发生,发展后期易造成关节活动受限以及截肢的可能性。到底强直性脊柱炎能治好吗?求医网特邀专家为您详细解析。", "http://www.qiuyi.cn/ 到底强直性脊柱炎能治好吗强直性脊柱炎是一种慢性炎症性疾病, 在当今的临床上面控制起来的难度很 大的,首先侵犯骶髂关节。"], "is_selected": false, "title": "到底强直性脊柱炎能治好吗_百度文库"}], "question": "强直性脊柱炎能治愈吗", "id": 58}
```
转换数据集的代码为
```
def json2tsv(_file_):
    label_map = {
        'Yes':1,
        'Depends':2,
        'No':0
        }
    if isinstance(_file_, str):
        data = pd.read_json(_file_, lines=True)
    else:
        data = _file_
    text_a = data.question.apply(lambda x: ''.join(x.split('\n')))
    text_b = data.answer.apply(lambda x: ''.join(x.split('\n')))
    df_data = pd.DataFrame()
    df_data['text_a'] = text_a
    df_data['text_b'] = text_b
    label = data.yesno_answer
    df_data['label'] = label
    df_data['label'] = df_data['label'].replace(label_map)
    return df_data
if __name__ == '__main__':
    df_train = json2tsv('data/train.json')
    df_test = json2tsv('data/test.json')
    df_dev  = json2tsv('data/dev.json')
    df_train.to_csv('data/train.tsv', sep='\t', index=0)
    df_test.to_csv('data/test.tsv', sep='\t', index=0)
    df_dev.to_csv('data/dev.tsv', sep='\t', index=0)
```
转换后
```
textA                textB               Label
强直性脊柱炎能治愈吗            这种疾病在临床上是可以治好的!但是一定要坚持早发现早治疗的原则，否则病情一旦加重，会有生命危险，治疗起来也相对困难。        1       
```

在本次比赛中我们采用目前文本分类任务中最先进的几个模型进行预训练。从而得到分类能力比较良好的类模型，之后对每个模型进行 k 折法，最后将 k 折法后的模型得到的概率进行融合得到预测结果，使用这种多模型的联合预测的 方式有效的提升了模型的鲁棒性和准确性，使得最终得以进入前五。但比赛中仍 存在很多改进得地方，可能需要尝试但是由于时间紧迫却没有去测试的东西，比如，从所给予的文本中重新预训练模型，表达能力是否会更好。进行更加细致的预处理，分数也许会提升一些。

下载预训练模型
```
from paddlepalm import downloader
downloader.ls('pretrain')
downloader.download('pretrain', 'ERNIE-v1-zh-base', './pretrain_models')
downloader.download('pretrain', 'RoBERTa-zh-base', './pretrain_models')
downloader.download('pretrain', 'RoBERTa-zh-large', './pretrain_models')
downloader.download('pretrain', 'BERT-zh-base', './pretrain_models')
```

In [ ]:
#终端访问work文件夹
cd work/
#设置训练模型，运行run.py即可
python run.py --model-name RoBERTa-zh-large --config-name roberta

In [ ]:
#设置训练好模型后，运行predict.py
python predict.py --model-name RoBERTa-zh-large --config-name roberta

# **6、实验结果**
对于参数的调节，我们主要针对学习率和训练的次数所优化，如图是我们进 行参数调节所得到的效果提升，因为时间较少，只选择了 RoBERTa 进行了参数调节，其他模型与 RoBERTa 所选最优参数相同。可以看出来当 epochs 选择 5，learning rate 选择 3e-5 时它的分数最高，所以最终模型的参数如此选择。

| 列数           | epochs    | learning rate | score | 
| -------------- | ------------ | -------- |-------- |
| 1    | 1        |3e-5     | 80.2| 
| 2    | 3         |3e-5        | 80.9| 
| 3    |5         |3e-5       |81.7 | 
| 4    | 7        | 3e-5        | 81.3| 
| 5     | 5         | 1e-5        | 81.2| 
| 6    | 5         | 5e-5        |81.6 | 


# **7、总结**
在数据增强方面由于时间比较紧张，只进行了一组扩增测试，在给出的问题相关文档中包含多组段落摘要和标题，标题和 question 是强相关的，段落摘要 与回答强相关，因此，可以借助相关文档中的段落摘要与标题对原数据集进行扩充。但是发现效果反而下降的厉害，可能是因为测试集中和训练集扩增的数据中 所存在的字段不同导致过拟合严重，分数大幅度下降。
在本次比赛中我们采用目前文本分类任务中最先进的今个模型进行预训练从而得到分类能力比较良好的类模型，之后对每个模型进行 k 折法，最后将 k 折法后的模型得到的概率进行融合得到预测结果，使用这种多模型的联合预测的 方式有效的提升了模型的鲁棒性和准确性，使得最终得以进入前五。但比赛中仍 存在很多改进得地方，可能需要尝试但是由于时间紧迫却没有去测试的东西，比 如，从所给予的文本中重新预训练模型，表达能力是否会更好。进行更加细致的预处理，分数也许会提升一些。

# 项目成员
白马非马 https://aistudio.baidu.com/aistudio/personalcenter/thirdview/274929

白鸟 https://aistudio.baidu.com/aistudio/personalcenter/thirdview/442836

嘻嘻qhy https://aistudio.baidu.com/aistudio/personalcenter/thirdview/452420